In [1]:
import pyterrier as pt
pt.init(tqdm='notebook')

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
import torch
torch.__version__

'1.10.0+cu113'

## load the index

In [3]:
import pyterrier_colbert
from pyterrier_colbert.ranking import *
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"
index=("/nfs/indices/colbert_passage","index_name3")
factory = pyterrier_colbert.ranking.ColBERTFactory(checkpoint, *index, )



Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Feb 11, 14:36:21] #> Loading model checkpoint.
[Feb 11, 14:36:21] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


/opt/conda/envs/colbert38/lib/python3.8/site-packages/torch/hub.py:513: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[Feb 11, 14:36:31] #> checkpoint['epoch'] = 0
[Feb 11, 14:36:31] #> checkpoint['batch'] = 44500


In [4]:
factory.faiss_index_on_gpu = False
nn_term = factory.nn_term(df=True)

[Feb 11, 14:36:32] #> Loading the FAISS index from /nfs/indices/colbert_passage/index_name3/ivfpq.faiss ..
[Feb 11, 14:37:04] #> Building the emb2pid mapping..
[Feb 11, 14:37:27] len(self.emb2pid) = 687989391
[Feb 11, 14:37:28] #> Building the emb2tid mapping..
687989391
Loading doclens


The following cell forces the ColBERT embeddings index to be loaded into memory. 

In [5]:
[x.mmap.shape[0] for x in factory._rrm().part_mmap]

Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/24 [00:00<?, ?shard/s]

[28603376,
 28935602,
 28997615,
 29001911,
 29033097,
 29094152,
 28649536,
 28723825,
 28856902,
 28970728,
 29018059,
 29025293,
 29029230,
 29065641,
 29474976,
 30061199,
 33096705,
 30056847,
 29293124,
 27428802,
 28026855,
 27215683,
 28375647,
 19954586]

## index stats

In [6]:
total = torch.sum(nn_term.lookup)
total

tensor(687989391)

In [7]:
len(factory)

8841823

In [8]:
total/len(factory)

tensor(77.8108)

## eval setup

In [9]:
%pip install --force-reinstall --no-deps git+https://github.com/terrierteam/ir_measures.git@runtime

  Cloning https://github.com/terrierteam/ir_measures.git (to revision runtime) to /tmp/pip-req-build-3gv2mv1p
  Running command git clone -q https://github.com/terrierteam/ir_measures.git /tmp/pip-req-build-3gv2mv1p
  Running command git checkout -b runtime --track origin/runtime
  Switched to a new branch 'runtime'
  Branch 'runtime' set up to track remote branch 'runtime' from 'origin'.
  Created wheel for ir-measures: filename=ir_measures-0.2.3-py3-none-any.whl size=54379 sha256=ed9cce06e033cd5ef86f3c6670f99712e036453662e04cecdc0c997ea793ce54
  Stored in directory: /tmp/pip-ephem-wheel-cache-atfw1vrg/wheels/77/f7/6a/b55aee2281a6d74fae948bdba4c5b80e95cd8b6b2001fbd3e3
Successfully built ir-measures
  Attempting uninstall: ir-measures
    Found existing installation: ir-measures 0.2.3
    Uninstalling ir-measures-0.2.3:
      Successfully uninstalled ir-measures-0.2.3
Note: you may need to restart the kernel to use updated packages.


In [40]:
def _AvgDocLen(qrels, run):
    from warnings import warn
    if 'doc_toks' not in run.columns:
        if len(run) > 0:
            warn("run of %d rows did not have doc_toks column; available columns: %s" % (len(run), str(run.columns)))
        else:
            warn("empty run did not have doc_toks column; available columns: %s" % str(run.columns))
        return 0
    return run['doc_toks'].apply(lambda row: row.shape[0]).mean()
    
import ir_measures
AvgDocLen = ir_measures.define_byquery(
    _AvgDocLen, 
    name="AvgDoclen")

from pyterrier.measures import *
MEASURES = [AP(rel=2)@1000, nDCG@10,nDCG@20,nDCG@100, RR(rel=2)@10,RR(rel=2),RR@10, RR ,R(rel=2)@1000,"mrt", AvgDocLen@1, AvgDocLen@10, AvgDocLen@100, AvgDocLen]


## "Original" baseline

In [12]:
baselineApprox = (
    (factory.ann_retrieve_score() % 1000) 
    >> factory.fetch_index_encodings(ids=True, verbose=False) 
    >> factory.scorer()
)

The following function does some of the grunt work for performing the experimental evaluations

In [41]:
def do_exp(pipelines, topics='2019', names=None, baseline=None, save_dir=None) -> pd.DataFrame:
    TQ = None
    topics_map = { '2019' : 'test-2019', '2020' : 'test-2020', 'dev' : 'dev.small' }
    TQ = (
       pt.get_dataset("msmarco_passage").get_topics(topics_map[topics]), 
       pt.get_dataset("msmarco_passage").get_qrels(topics_map[topics])
    )
    if save_dir is not None:
        save_dir = save_dir % str(topics)
        os.makedirs(save_dir, exist_ok=True)
    df = pt.Experiment(
        pipelines, 
        *TQ,
        MEASURES,
        batch_size=20,
        filter_by_qrels=True,
        verbose=True,
        names=names,
        save_dir=save_dir
    )
    if save_dir:
        df.to_pickle(save_dir + "/results_df.pkl")
        
    return df

This loop provides the results files and eval results for the '2019', '2020', 'dev' topics sets

In [85]:
all_res = [
    do_exp(
        [baselineApprox], 
        names=['approx_baseline'], 
        topics=t,
        save_dir='/nfs/staticPruning/results/repro/baseline_%s/'
    ) for t in ['2019', '2020', 'dev']
]
pd.concat(all_res)

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen,mrt
0,approx_baseline,0.405597,0.692943,0.649851,0.584233,0.853488,0.853659,0.939922,0.939922,0.695360,67.883721,76.690698,79.226279,77.001302,645.998443
0,approx_baseline,0.455459,0.689205,0.657368,0.611870,0.850000,0.852646,0.922840,0.925485,0.750889,68.981481,74.324074,78.199815,76.993000,NaN
0,approx_baseline,0.000000,0.401613,0.426102,0.454005,0.000000,0.000000,0.341710,0.352399,0.000000,69.069054,74.954656,78.020663,76.502952,NaN


## uniform IDF pruning

We now design the implementations of the uniform IDF static pruning approach

In [42]:
def uniform_blacklisted_tokens_transformer(blacklist, verbose=False, zero=False, gpu=torch.cuda.is_available()) -> TransformerBase:
    """
    Remove tokens and their embeddings from the document dataframe
    input: qid, query_embs, docno, doc_embs, doc_toks
    output: qid, query_embs, docno, doc_embs, doc_toks
    
    The blacklist parameters must contain a list of tokenids that should be removed
    """
    import pyterrier as pt
    import torch
    import numpy as np
    
    assert pt.started(), 'PyTerrier must be started'
    
    if gpu: 
        blacklist = torch.Tensor(blacklist).cuda()
    else:
        blacklist = torch.Tensor(blacklist)
        
    pt.tqdm.pandas()
    
    if verbose: print(f'Blacklist composed of {len(blacklist)} elements.')
        
    def _prune_remove_gpu(row):
        tokens = row['doc_toks'].cuda()
        embeddings = row['doc_embs'].cuda()

        # create the 1-D mask
        final_mask = torch.all(tokens[None, : ] != blacklist[:, None], axis=0)

        # apply the mask to the tokens
        row['doc_toks'] = tokens[final_mask]

        # pad the 1-d mask
        row_embs_size = embeddings.size()
        mask_1d = torch.cat((final_mask, torch.ones(row_embs_size[0] - final_mask.size()[0], dtype=torch.bool).cuda()))

        # apply the padded 1-d mask
        row['doc_embs'] = embeddings[mask_1d, : ]
        return row

    def _prune_remove(row):
        tokens = row['doc_toks']
        embeddings = row['doc_embs']

        # create the 1-D mask
        final_mask = torch.all(tokens[None, : ] != blacklist[:, None], axis=0)

        # apply the mask to the tokens
        row['doc_toks'] = tokens[final_mask]

        # pad the 1-d mask
        row_embs_size = embeddings.size()
        mask_1d = torch.cat((final_mask, torch.ones(row_embs_size[0] - final_mask.size()[0], dtype=torch.bool)))

        # apply the padded 1-d mask
        row['doc_embs'] = embeddings[mask_1d, : ]
        return row
    
    def _prune_gpu_zero(row):
        tokens = row['doc_toks'].cuda()
        embeddings = row['doc_embs'].cuda()
        row_embs_size = embeddings.size()
        tokens_size = tokens.size()[0]
        
        # create the 1-D mask
        final_mask = torch.zeros(row_embs_size[0], dtype=torch.bool)
        final_mask[0:tokens_size] = torch.any(tokens[None, :] == blacklist[:, None], axis=0)
        
        # apply the mask
        row['doc_toks'][final_mask[0:tokens_size]] = 0
        row['doc_embs'][final_mask, :] = 0 
        return row
        
    def _prune_zero(row):
        tokens = row['doc_toks']
        embeddings = row['doc_embs']
        row_embs_size = embeddings.size()
        tokens_size = tokens.size()[0]
                                        
        # create the 1-D mask
        final_mask = torch.zeros(row_embs_size[0], dtype=torch.bool)
        final_mask[0:tokens_size] = torch.any(tokens[None, :] == blacklist[:, None], axis=0)
            
        # apply the mask
        row['doc_toks'][final_mask[0:tokens_size]] = 0
        row['doc_embs'][final_mask, :] = 0
        return row
    
    if zero:
        prune_function = _prune_gpu_zero if gpu else _prune_zero
    else:
        prune_function = _prune_remove_gpu if gpu else _prune_remove

    def _apply(df):
        if verbose:
            df = df.progress_apply(prune_function, axis=1)
        else:
            df = df.apply(prune_function, axis=1)
        return df

    return pt.apply.generic(_apply)

def get_uniform_pipeline(factory, blacklist, k1=1000):
    pipeline = (
        factory.query_encoder()
        >> (factory.ann_retrieve_score(query_encoded=True) % k1)
        >> factory.fetch_index_encodings(ids=True, verbose=False)
        >> uniform_blacklisted_tokens_transformer(blacklist, verbose=False, zero=False)
        >> factory.scorer(verbose=False)
        #>> pt.apply.doc_embs(drop=True)
        #>> pt.apply.query_embs(drop=True)
    )
    return pipeline

In [43]:
from pyterrier_colbert.faiss_term_index import FaissNNTerm
def get_tid_ordered_by_idf(faiss_nn_term : FaissNNTerm, verbose=False):
    import math
    assert hasattr(faiss_nn_term, 'dfs'), 'FaissNNTerm class must be initialized with dfs=True'

    vocabulary = faiss_nn_term.tok.get_vocab()
    n_docs = faiss_nn_term.num_docs

    if verbose:
        print(f'Number of docs: {n_docs}')
        print(f'Vocabulary Length: {len(vocabulary)}')

    tokens = []
    for token in vocabulary:
        tid = vocabulary[token]
        df = faiss_nn_term.getDF_by_id(tid)
        if(df != 0):
            idf = math.log(n_docs/(df + 1), 10)
            tokens.append((tid, idf))

    # Remove items with 0 document frequency
    if verbose: print("Token length (without 0 df elements):", len(tokens))
    # order by inverse document frequency
    ordered_tokens = sorted(tokens, key= lambda pair: pair[1])
    final_token_list = [_id for _id, _ in ordered_tokens]
    return final_token_list


DROPS corresponds to the $\tau_u$ in the paper

In [44]:
DROPS=[1, 10, 25, 50, 100, 1000, 2000, 3000]


tid_ordered_by_idf = get_tid_ordered_by_idf(nn_term)

#we always start from 3 to ignore the CLS, D tokens etc.
tid_ordered_by_idf[3:13]


[1012, 1996, 1010, 1997, 1998, 1037, 2000, 1999, 2003, 2005]

In [87]:
idf_2019 = do_exp(
    [get_uniform_pipeline(factory, tid_ordered_by_idf[3:x+3]) for x in DROPS ], 
    topics='2019',
    names=["uniform_%d" % x for x in DROPS ],
    save_dir="/nfs/staticPruning/results/repro/uniform_%s/"
)
idf_2019

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_1,0.405961,0.694331,0.651018,0.584537,0.853488,0.853662,0.941860,0.941860,0.69536,64.558140,72.981395,75.453256,73.233512
1,uniform_10,0.404652,0.695357,0.652229,0.582998,0.865116,0.865290,0.939922,0.939922,0.69536,50.093023,57.802326,59.156512,57.486000
2,uniform_25,0.404306,0.686533,0.648563,0.581893,0.865116,0.865295,0.939922,0.939922,0.69536,44.279070,51.158140,52.490000,50.999023
3,uniform_50,0.403207,0.686432,0.649681,0.580809,0.853488,0.853674,0.939922,0.939922,0.69536,40.906977,46.897674,48.471395,47.087442
4,uniform_100,0.402592,0.688892,0.649656,0.579712,0.845736,0.845922,0.932171,0.932171,0.69536,37.651163,43.348837,44.694186,43.435581
5,uniform_1000,0.378553,0.647829,0.611177,0.540572,0.843023,0.844743,0.947674,0.949042,0.69536,25.046512,28.367442,27.860930,27.453767
6,uniform_2000,0.333716,0.613761,0.565548,0.504106,0.843798,0.846744,0.937984,0.939535,0.69536,19.418605,20.904651,21.237209,20.870930
7,uniform_3000,0.315022,0.568992,0.520151,0.471269,0.814175,0.817290,0.879291,0.883096,0.69536,15.279070,16.951163,17.212093,16.873256


In [88]:
idf_2020 = do_exp(
    [get_uniform_pipeline(factory, tid_ordered_by_idf[3:x+3]) for x in DROPS ],
    topics='2020',
    names=["uniform_%d" % x for x in DROPS ],
    save_dir="/nfs/staticPruning/results/repro/uniform_%s/"
)
idf_2020

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_1,0.455497,0.688954,0.657722,0.611513,0.850000,0.852557,0.922840,0.925397,0.750889,65.759259,70.692593,74.533333,73.139389
1,uniform_10,0.457219,0.690652,0.656046,0.611886,0.857716,0.860565,0.922840,0.925689,0.750889,53.055556,55.650000,58.687407,57.559870
2,uniform_25,0.460114,0.690334,0.659018,0.610760,0.858951,0.860634,0.924691,0.926375,0.750889,46.166667,49.550000,52.143333,51.190611
3,uniform_50,0.457625,0.691289,0.658722,0.612326,0.850485,0.852169,0.925485,0.927169,0.750889,43.129630,45.505556,47.872037,46.984296
4,uniform_100,0.457333,0.688052,0.657896,0.612444,0.850044,0.851728,0.925485,0.927169,0.750889,39.462963,41.918519,43.905185,43.076444
5,uniform_1000,0.421563,0.663536,0.605718,0.574324,0.815608,0.815608,0.950617,0.950617,0.750889,24.111111,27.203704,28.050556,27.133370
6,uniform_2000,0.383755,0.593645,0.556330,0.529415,0.754652,0.758675,0.905710,0.906867,0.750889,18.759259,20.633333,21.314815,20.536315
7,uniform_3000,0.368490,0.570477,0.544948,0.513365,0.753131,0.754365,0.889065,0.889065,0.750889,16.055556,17.098148,17.577963,16.850463


In [92]:
idf_dev = do_exp(
    [get_uniform_pipeline(factory, tid_ordered_by_idf[3:x+3]) for x in DROPS ],
    topics='dev',
    names=["uniform_%d" % x for x in DROPS ],
    save_dir="/nfs/staticPruning/results/repro/uniform_%s/"
)
idf_dev

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_1,0.0,0.400824,0.425659,0.453466,0.0,0.0,0.340971,0.351744,0.0,65.435817,71.118438,74.046887,72.612341
1,uniform_10,0.0,0.399779,0.425007,0.452798,0.0,0.0,0.340252,0.351191,0.0,52.039971,56.304799,58.404444,57.268624
2,uniform_25,0.0,0.398953,0.424126,0.452151,0.0,0.0,0.339520,0.350502,0.0,46.336676,49.993123,51.876050,50.917969
3,uniform_50,0.0,0.398091,0.423260,0.451413,0.0,0.0,0.338807,0.349741,0.0,42.690115,45.927808,47.573274,46.675498
4,uniform_100,0.0,0.394788,0.420355,0.448464,0.0,0.0,0.335730,0.346737,0.0,39.240258,42.217837,43.711490,42.844141
5,uniform_1000,0.0,0.359918,0.383584,0.415319,0.0,0.0,0.303860,0.314943,0.0,25.271347,26.715774,27.406635,26.782411
6,uniform_2000,0.0,0.333851,0.357203,0.390378,0.0,0.0,0.280669,0.291872,0.0,19.515616,20.455186,20.911254,20.361803
7,uniform_3000,0.0,0.313844,0.336802,0.370560,0.0,0.0,0.263599,0.274841,0.0,16.043696,16.724212,17.021218,16.539694


The next 2 cells ran some additional datapoints for the uniform pruning, and copied them into the files.

In [46]:
from IPython.display import display
for t in ['2019', '2020', 'dev']:
    idf = do_exp(
        [get_uniform_pipeline(factory, tid_ordered_by_idf[3:x+3]) for x in [250,500,750] ], 
        topics=t,
        names=["uniform_%d" % x for x in [250,500,750] ],
        save_dir="/nfs/staticPruning/results/repro/supplement_uniform_%s/"
    )
    display(idf)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_250,0.388886,0.657279,0.624054,0.566356,0.807586,0.809249,0.936047,0.937271,0.695251,32.209302,38.093023,38.870233,38.075884
1,uniform_500,0.375526,0.643821,0.607205,0.541373,0.802326,0.804457,0.924419,0.926208,0.695251,30.209302,33.630233,33.715814,33.177233
2,uniform_750,0.384139,0.648789,0.611416,0.544533,0.815891,0.817790,0.936047,0.937597,0.695251,27.720930,30.618605,30.358140,29.916605


pt.Experiment:   0%|          | 0/9 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_250,0.454676,0.690524,0.652513,0.607058,0.860516,0.860516,0.943673,0.943673,0.754593,33.185185,36.896296,38.307407,37.609278
1,uniform_500,0.444291,0.681340,0.631334,0.596277,0.844753,0.844753,0.958333,0.958333,0.754593,29.462963,32.405556,33.394074,32.691685
2,uniform_750,0.436530,0.679331,0.625340,0.587080,0.846296,0.846296,0.950617,0.950617,0.754593,26.537037,29.579630,30.361481,29.504944


pt.Experiment:   0%|          | 0/1047 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_250,0.0,0.387861,0.412696,0.441257,0.0,0.0,0.328845,0.339812,0.0,34.629513,37.015287,38.195284,37.384855
1,uniform_500,0.0,0.373959,0.398772,0.428734,0.0,0.0,0.316276,0.327443,0.0,30.188682,32.125129,33.118317,32.445901
2,uniform_750,0.0,0.365723,0.389993,0.420990,0.0,0.0,0.308556,0.319766,0.0,27.420057,29.093968,29.898242,29.234465


In [53]:
import shutil
for t in ['2019', '2020', 'dev']:
    add_dir = "/nfs/staticPruning/results/repro/supplement_uniform_%s/" % t
    main_dir = "/nfs/staticPruning/results/repro/uniform_%s/" % t
    newdf = pd.concat([
        pd.read_pickle(add_dir + "results_df.pkl"),
        pd.read_pickle(main_dir + "results_df.pkl"),
    ]).iloc[[3,4,5,6,7,0,1,2,8,9,10]]
    display(newdf)
    os.rename(main_dir + "results_df.pkl", main_dir + "results_df.backup.pkl")
    newdf.to_pickle(main_dir + "results_df.pkl")
    for x in [250,500,750]:
        shutil.copyfile(add_dir + "uniform_%d.res.gz" %x, main_dir+ "uniform_%d.res.gz" %x)

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_1,0.405961,0.694331,0.651018,0.584537,0.853488,0.853662,0.941860,0.941860,0.695360,64.558140,72.981395,75.453256,73.233512
1,uniform_10,0.404652,0.695357,0.652229,0.582998,0.865116,0.865290,0.939922,0.939922,0.695360,50.093023,57.802326,59.156512,57.486000
2,uniform_25,0.404306,0.686533,0.648563,0.581893,0.865116,0.865295,0.939922,0.939922,0.695360,44.279070,51.158140,52.490000,50.999023
3,uniform_50,0.403207,0.686432,0.649681,0.580809,0.853488,0.853674,0.939922,0.939922,0.695360,40.906977,46.897674,48.471395,47.087442
4,uniform_100,0.402592,0.688892,0.649656,0.579712,0.845736,0.845922,0.932171,0.932171,0.695360,37.651163,43.348837,44.694186,43.435581
0,uniform_250,0.388886,0.657279,0.624054,0.566356,0.807586,0.809249,0.936047,0.937271,0.695251,32.209302,38.093023,38.870233,38.075884
1,uniform_500,0.375526,0.643821,0.607205,0.541373,0.802326,0.804457,0.924419,0.926208,0.695251,30.209302,33.630233,33.715814,33.177233
2,uniform_750,0.384139,0.648789,0.611416,0.544533,0.815891,0.817790,0.936047,0.937597,0.695251,27.720930,30.618605,30.358140,29.916605
5,uniform_1000,0.378553,0.647829,0.611177,0.540572,0.843023,0.844743,0.947674,0.949042,0.695360,25.046512,28.367442,27.860930,27.453767
6,uniform_2000,0.333716,0.613761,0.565548,0.504106,0.843798,0.846744,0.937984,0.939535,0.695360,19.418605,20.904651,21.237209,20.870930


,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_1,0.455497,0.688954,0.657722,0.611513,0.850000,0.852557,0.922840,0.925397,0.750889,65.759259,70.692593,74.533333,73.139389
1,uniform_10,0.457219,0.690652,0.656046,0.611886,0.857716,0.860565,0.922840,0.925689,0.750889,53.055556,55.650000,58.687407,57.559870
2,uniform_25,0.460114,0.690334,0.659018,0.610760,0.858951,0.860634,0.924691,0.926375,0.750889,46.166667,49.550000,52.143333,51.190611
3,uniform_50,0.457625,0.691289,0.658722,0.612326,0.850485,0.852169,0.925485,0.927169,0.750889,43.129630,45.505556,47.872037,46.984296
4,uniform_100,0.457333,0.688052,0.657896,0.612444,0.850044,0.851728,0.925485,0.927169,0.750889,39.462963,41.918519,43.905185,43.076444
0,uniform_250,0.454676,0.690524,0.652513,0.607058,0.860516,0.860516,0.943673,0.943673,0.754593,33.185185,36.896296,38.307407,37.609278
1,uniform_500,0.444291,0.681340,0.631334,0.596277,0.844753,0.844753,0.958333,0.958333,0.754593,29.462963,32.405556,33.394074,32.691685
2,uniform_750,0.436530,0.679331,0.625340,0.587080,0.846296,0.846296,0.950617,0.950617,0.754593,26.537037,29.579630,30.361481,29.504944
5,uniform_1000,0.421563,0.663536,0.605718,0.574324,0.815608,0.815608,0.950617,0.950617,0.750889,24.111111,27.203704,28.050556,27.133370
6,uniform_2000,0.383755,0.593645,0.556330,0.529415,0.754652,0.758675,0.905710,0.906867,0.750889,18.759259,20.633333,21.314815,20.536315


,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,uniform_1,0.0,0.400824,0.425659,0.453466,0.0,0.0,0.340971,0.351744,0.0,65.435817,71.118438,74.046887,72.612341
1,uniform_10,0.0,0.399779,0.425007,0.452798,0.0,0.0,0.340252,0.351191,0.0,52.039971,56.304799,58.404444,57.268624
2,uniform_25,0.0,0.398953,0.424126,0.452151,0.0,0.0,0.339520,0.350502,0.0,46.336676,49.993123,51.876050,50.917969
3,uniform_50,0.0,0.398091,0.423260,0.451413,0.0,0.0,0.338807,0.349741,0.0,42.690115,45.927808,47.573274,46.675498
4,uniform_100,0.0,0.394788,0.420355,0.448464,0.0,0.0,0.335730,0.346737,0.0,39.240258,42.217837,43.711490,42.844141
0,uniform_250,0.0,0.387861,0.412696,0.441257,0.0,0.0,0.328845,0.339812,0.0,34.629513,37.015287,38.195284,37.384855
1,uniform_500,0.0,0.373959,0.398772,0.428734,0.0,0.0,0.316276,0.327443,0.0,30.188682,32.125129,33.118317,32.445901
2,uniform_750,0.0,0.365723,0.389993,0.420990,0.0,0.0,0.308556,0.319766,0.0,27.420057,29.093968,29.898242,29.234465
5,uniform_1000,0.0,0.359918,0.383584,0.415319,0.0,0.0,0.303860,0.314943,0.0,25.271347,26.715774,27.406635,26.782411
6,uniform_2000,0.0,0.333851,0.357203,0.390378,0.0,0.0,0.280669,0.291872,0.0,19.515616,20.455186,20.911254,20.361803


## IDF document-centric pruning 

We now turn to implement the transformers for the document-centric pruning.

In [56]:

from typing import Tuple, Sequence

def prune_sorted_natural(x, *Ys):
    assert len(x.shape)==1
    
    indices = torch.sort(x, descending=True).indices

    # concatenate neg and pos sides
    rtr = [
        x[indices]
    ]
    
    # apply same slicing for other arrays
    for y in Ys:        
        y_sorted = y[indices]
        rtr.append(y_sorted)
    return rtr

def doc_centric_idf(tokenids : torch.Tensor, embs : torch.Tensor, num_prune) ->  Tuple[Sequence[int], torch.Tensor]:
    """
    Returns the tokenids and embs after remvoing num_prune embeddings
    """
       
    length = len(tokenids)
    newmask = torch.BoolTensor(length)
    newmask[:] = True
    embs = embs[0:length, :]
    
    dfs = nn_term.lookup[tokenids]
    
    #sort the embeddings according to pruning power
    sorted_dfs, pos = prune_sorted_natural(
        dfs, 
        torch.LongTensor(list(range(length)))
    )

    # prune the requisite numbers
    newmask[pos[0:num_prune]] = False
    
    return tokenids[newmask], embs[newmask,:]


def prune_doccentric_transformerH0(factory, topK, active_ids=None, min_obs=0):
    assert active_ids is None 
    #if active_ids is None:
    #    active_ids = list(range(nn_term.lookup.shape[0]))
        
        
    
    def _prune(df):
        #pt.tqdm.pandas(desc='prune_doccentric')
        #progress_apply
        df[["doc_toks", "doc_embs"]] = df.apply(
            lambda row: doc_centric_idf(row.doc_toks, row.doc_embs, topK),
            #lambda row: tc.ignore_prune(row.doc_toks, row.doc_embs, topK, pruning='global', blacklist=blacklist), 
            result_type="expand", 
            axis=1)
        return df
    return pt.apply.generic(_prune)

def prune_doccentric_pipeH0(topK, approx=True):
    first_pass = (factory.ann_retrieve_score() % 1000) if approx else factory.set_retrieve()
    return ( first_pass
        >> factory.fetch_index_encodings(ids=True, verbose=False) 
        >> prune_doccentric_transformerH0(factory, topK)
        >> factory.scorer(verbose=False))


check that the document-centric pruning works.

In [58]:
prune_doccentric_pipeH0(10).search("chemical reactions")[['qid', 'docno']]

,qid,docno
11,1,7605154
8,1,5453527
121,1,1833252
19,1,7575552
52,1,1525431
...,...,...
1701,1,2130934
1889,1,5812915
1732,1,6823853
1995,1,976146


now conduct the experiments for document-centric pruning. 

`DOC_CENTRIC_THRESHOLDS` corresponds to $\tau_d$ in the paper.

In [62]:
DOC_CENTRIC_THRESHOLDS=[1,3,5,8,10,15,20,25,30]

In [76]:
doccentric_2019 = do_exp(
    [prune_doccentric_pipeH0(x) for x in DOC_CENTRIC_THRESHOLDS], 
    topics='2019',
    names=["doc_centric_%d" % x for x in DOC_CENTRIC_THRESHOLDS],
    save_dir="/nfs/staticPruning/results/repro/doccentric_%s/"
)
doccentric_2019

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,doc_centric_1,0.405220,0.692956,0.649440,0.584247,0.853488,0.853659,0.69536,66.883721,75.690698,78.282326,76.001302
1,doc_centric_3,0.408684,0.698561,0.652257,0.586583,0.865116,0.865287,0.69536,66.581395,74.383721,76.500930,74.001302
2,doc_centric_5,0.411256,0.701147,0.654258,0.586850,0.865116,0.865292,0.69536,64.581395,72.330233,74.726512,72.001302
3,doc_centric_8,0.400595,0.694253,0.647600,0.580938,0.865116,0.865294,0.69536,61.813953,69.530233,71.956512,69.001302
4,doc_centric_10,0.400391,0.690964,0.645815,0.581171,0.868992,0.869171,0.69536,59.744186,68.290698,70.318837,67.001302
5,doc_centric_15,0.401108,0.691849,0.651776,0.581463,0.868992,0.869188,0.69536,56.488372,65.255814,66.801860,62.001395
6,doc_centric_20,0.385735,0.680996,0.636953,0.569487,0.863178,0.863398,0.69536,54.674419,63.872093,63.850698,57.002837
7,doc_centric_25,0.370902,0.658937,0.626693,0.555300,0.839922,0.840106,0.69536,54.906977,61.834884,61.062093,52.013651
8,doc_centric_30,0.346869,0.641218,0.602858,0.534962,0.787597,0.788548,0.69536,55.883721,59.676744,58.417674,47.050721


In [78]:
doccentric_2020 = do_exp(
    [prune_doccentric_pipeH0(x) for x in DOC_CENTRIC_THRESHOLDS], 
    topics='2020',
    names=["doc_centric_%d" % x for x in DOC_CENTRIC_THRESHOLDS],
    save_dir="/nfs/staticPruning/results/repro/doccentric_%s/"
)
doccentric_2020

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,doc_centric_1,0.455460,0.689148,0.657294,0.611809,0.850000,0.852557,0.750889,67.981481,73.320370,77.244815,75.993000
1,doc_centric_3,0.456174,0.689630,0.658862,0.611955,0.850000,0.852557,0.750889,66.129630,71.200000,75.377778,73.993000
2,doc_centric_5,0.456169,0.689525,0.660041,0.611555,0.850000,0.852557,0.750889,64.129630,69.735185,73.578704,71.993000
3,doc_centric_8,0.454840,0.691973,0.658627,0.611849,0.850000,0.852557,0.750889,61.500000,67.179630,71.000556,68.993000
4,doc_centric_10,0.457086,0.694662,0.659177,0.612022,0.850000,0.852412,0.750889,59.500000,65.453704,69.422778,66.993000
5,doc_centric_15,0.451205,0.692655,0.652491,0.608864,0.838889,0.840046,0.750889,59.888889,63.144444,66.421481,61.993185
6,doc_centric_20,0.450207,0.689134,0.652400,0.605126,0.847531,0.849074,0.750889,58.185185,61.144444,63.957222,56.997167
7,doc_centric_25,0.438444,0.675905,0.642909,0.594756,0.833025,0.833025,0.750889,56.351852,58.292593,61.602593,52.018222
8,doc_centric_30,0.423784,0.666966,0.632528,0.584582,0.835391,0.836366,0.750889,51.962963,56.733333,58.869630,47.066019


In [91]:
doccentric_dev = do_exp(
    [prune_doccentric_pipeH0(x) for x in DOC_CENTRIC_THRESHOLDS], 
    topics='dev',
    names=["doc_centric_%d" % x for x in DOC_CENTRIC_THRESHOLDS],
    save_dir="/nfs/staticPruning/results/repro/doccentric_%s/"
)
doccentric_dev

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,doc_centric_1,0.0,0.401628,0.426198,0.454013,0.0,0.0,0.341719,0.352404,0.0,68.102006,73.984928,77.043357,75.502952
1,doc_centric_3,0.0,0.401501,0.425715,0.453846,0.0,0.0,0.341451,0.352113,0.0,66.298567,72.158453,75.169029,73.502952
2,doc_centric_5,0.0,0.401206,0.425598,0.453503,0.0,0.0,0.341014,0.351669,0.0,64.690115,70.373238,73.296202,71.502952
3,doc_centric_8,0.0,0.401186,0.425374,0.453416,0.0,0.0,0.340853,0.351475,0.0,62.813467,67.863926,70.646749,68.502958
4,doc_centric_10,0.0,0.399992,0.424304,0.452097,0.0,0.0,0.339449,0.350034,0.0,61.748138,66.401862,69.049629,66.502966
5,doc_centric_15,0.0,0.392814,0.417930,0.446841,0.0,0.0,0.333808,0.344826,0.0,60.719628,64.008983,65.844202,61.503098
6,doc_centric_20,0.0,0.384567,0.410787,0.439235,0.0,0.0,0.326755,0.338073,0.0,60.881948,62.390616,63.108341,56.505853
7,doc_centric_25,0.0,0.370700,0.395180,0.425006,0.0,0.0,0.313317,0.324463,0.0,60.200430,60.563883,60.232650,51.521634
8,doc_centric_30,0.0,0.354905,0.379065,0.408670,0.0,0.0,0.301130,0.312367,0.0,59.107450,58.461304,57.304550,46.563818


# uniform idf index generation

We now turn to creating real indices. After index generation has completed, the faiss_index.py script within ColBERT must be executed, using the `--sample 0.05` setting.

In [30]:
def prune_index_to_disk_by_idf(factory, new_index_path, num_tokens, flush):
    import torch
    
    removed_tids = get_tid_ordered_by_idf(factory.nn_term(), verbose=False)[3:num_tokens+3]
    #removed_tids = torch.Tensor(removed_tids)
    removed_tids = set(removed_tids)
    
    import os, torch
    embs_to_save = []
    ids_to_save = []
    shard=0
    shard_size=0
    
    def _save_shard(shard, all_embs, all_ids):
        import ujson
        torch.save(torch.cat(all_embs), os.path.join(new_index_path, "%d.pt" % shard))
        torch.save(torch.cat(all_ids), os.path.join(new_index_path, "%d.tokenids" % shard))
        doclens_path = os.path.join(new_index_path, "doclens.%d.json" % shard)
        doclens = [d.size(0) for d in all_embs]
        with open(doclens_path, 'w') as output_doclens:
            ujson.dump(doclens, output_doclens)
        
    
    mask = torch.BoolTensor(180)
    mask[:] = False
    numdocs = len(factory)

    for pid in pt.tqdm(range(numdocs), total=numdocs, unit='d'):        
        ids = factory.nn_term().get_tokens_for_doc(pid)
        num_toks = ids.size(0)
        embs = factory._rrm().get_embedding(pid).type(torch.float16)
        embs = embs[0:num_toks,:]
        
        mask[0:num_toks] = torch.BoolTensor([x.item() in removed_tids for x in ids]) # torch.isin(ids, removed_tids)
        #This is 5x faster says Antonio
        #mask[0:num_tokens] = torch.any(ids[None, : ] == removed_tids[:, None], axis=0)

        newids = ids[~mask[:num_toks]]
        newembs = embs[~mask[:num_toks], : ]
        
        #assert newids.shape[0] > 0, newids.shape
        mask[:]  = False
        
        embs_to_save.append(newembs)
        ids_to_save.append(newids)
        shard_size += 1
        
        if shard_size == flush:
            print("Flushing shard %d" % shard)
            _save_shard(shard, embs_to_save, ids_to_save)
            shard += 1
            shard_size = 0
            embs_to_save=[]
            ids_to_save=[]
            print("Done")
            
    if shard_size > 0:
        _save_shard(shard, embs_to_save, ids_to_save)

In [31]:
import os
os.makedirs("/nfsindices/staticPrunedIndices/repro_paper_SIGIR2022/idf_uniform_100", exist_ok=True)
prune_index_to_disk_by_idf(
    factory, 
    "/nfsindices/staticPrunedIndices/repro_paper_SIGIR2022/idf_uniform_100", 
    100, 
    300_000)

  0%|          | 0/8841823 [00:00<?, ?d/s]

Flushing shard 0
Done
Flushing shard 1
Done
Flushing shard 2
Done
Flushing shard 3
Done
Flushing shard 4
Done
Flushing shard 5
Done
Flushing shard 6
Done
Flushing shard 7
Done
Flushing shard 8
Done
Flushing shard 9
Done
Flushing shard 10
Done
Flushing shard 11
Done
Flushing shard 12
Done
Flushing shard 13
Done
Flushing shard 14
Done
Flushing shard 15
Done
Flushing shard 16
Done
Flushing shard 17
Done
Flushing shard 18
Done
Flushing shard 19
Done
Flushing shard 20
Done
Flushing shard 21
Done
Flushing shard 22
Done
Flushing shard 23
Done
Flushing shard 24
Done
Flushing shard 25
Done
Flushing shard 26
Done
Flushing shard 27
Done
Flushing shard 28
Done


# stopwords removed index generation

The file `stopword-list.txt` corresponds to Terrier's stopwords file.

In [1]:
if not os.path.exists("stopword-list.txt"):
    !wget "https://raw.githubusercontent.com/terrier-org/terrier-core/5.x/modules/core/src/main/resources/stopword-list.txt"

stops=[]
with open("stopword-list.txt") as f:
    for l in f:
        stops.append(l.strip())

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
stop_ids = [x for x in tokenizer.convert_tokens_to_ids(stops) if x != 100]

In [3]:
tokenizer.convert_ids_to_tokens([1012, 1996, 1010, 1997, 1998, 1037, 2000, 1999, 2003, 2005])

['.', 'the', ',', 'of', 'and', 'a', 'to', 'in', 'is', 'for']

In [33]:
def prune_index_to_disk_by_stops(factory, new_index_path, flush):
    import torch
    
    removed_tids = stop_ids
    #removed_tids = torch.Tensor(removed_tids)
    removed_tids = set(removed_tids)
    
    import os, torch
    embs_to_save = []
    ids_to_save = []
    shard=0
    shard_size=0
    
    def _save_shard(shard, all_embs, all_ids):
        import ujson
        torch.save(torch.cat(all_embs), os.path.join(new_index_path, "%d.pt" % shard))
        torch.save(torch.cat(all_ids), os.path.join(new_index_path, "%d.tokenids" % shard))
        doclens_path = os.path.join(new_index_path, "doclens.%d.json" % shard)
        doclens = [d.size(0) for d in all_embs]
        with open(doclens_path, 'w') as output_doclens:
            ujson.dump(doclens, output_doclens)
        
    
    mask = torch.BoolTensor(180)
    mask[:] = False
    numdocs = len(factory)

    for pid in pt.tqdm(range(numdocs), total=numdocs, unit='d'):        
        ids = factory.nn_term().get_tokens_for_doc(pid)
        num_toks = ids.size(0)
        embs = factory._rrm().get_embedding(pid).type(torch.float16)
        embs = embs[0:num_toks,:]
        
        mask[0:num_toks] = torch.BoolTensor([x.item() in removed_tids for x in ids]) # torch.isin(ids, removed_tids)
        #This is 5x faster says Antonio
        #mask[0:num_tokens] = torch.any(ids[None, : ] == removed_tids[:, None], axis=0)

        newids = ids[~mask[:num_toks]]
        newembs = embs[~mask[:num_toks], : ]
        
        #assert newids.shape[0] > 0, newids.shape
        mask[:]  = False
        
        embs_to_save.append(newembs)
        ids_to_save.append(newids)
        shard_size += 1
        
        if shard_size == flush:
            print("Flushing shard %d" % shard)
            _save_shard(shard, embs_to_save, ids_to_save)
            shard += 1
            shard_size = 0
            embs_to_save=[]
            ids_to_save=[]
            print("Done")
            
    if shard_size > 0:
        _save_shard(shard, embs_to_save, ids_to_save)

In [ ]:
import os
os.makedirs("/nfsindices/staticPrunedIndices/repro_paper_SIGIR2022/stops", exist_ok=True)
prune_index_to_disk_by_stops(
    factory, 
    "/nfsindices/staticPrunedIndices/repro_paper_SIGIR2022/stops",
    300_000)

  0%|          | 0/8841823 [00:00<?, ?d/s]

Flushing shard 0
Done
Flushing shard 1
Done
Flushing shard 2
Done
Flushing shard 3
Done
Flushing shard 4
Done
Flushing shard 5
Done
Flushing shard 6
Done
Flushing shard 7
Done
Flushing shard 8
Done
Flushing shard 14
Done


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

